In [1]:
from utils import non_weather_training_columns_list, with_weather_training_columns_list
import pandas as pd
import numpy as np
import pickle 

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_parquet("../data/silver/training_dataset.parquet")

In [3]:
def forecast_next_24_hours_output_flow_rate(input_df, year, month, day, hour):
    
    timestamp = pd.Timestamp(year=year, month=month, day=day, hour=hour)
    input_df = input_df[input_df["timestamp"] == timestamp]
    print(input_df.target_1)
    
    X = input_df[non_weather_training_columns_list]
    X_weather = input_df[with_weather_training_columns_list]
    
    predictions = {}
    for i in range(1, 25):
        print(f"Loading model xgb_{i}h.pkl to predict target_{i}")
        model = pickle.load(open(f"../models/xgb_{i}h.pkl", "rb"))
        predictions[f'target_{i}'] = model.predict(X)
        print(f"predicted target_{i} = {predictions[f'target_{i}']}")
    
    
    predictions_with_weather = {}
    for i in range(1, 25):
        print(f"Loading model xgb_with_weather_{i}h.pkl to predict target_{i}")
        model = pickle.load(open(f"../models/xgb_with_weather_{i}h.pkl", "rb"))
        predictions_with_weather[f'target_{i}'] = model.predict(X_weather)
        print(f"predicted target_{i} = {predictions_with_weather[f'target_{i}']}")
    
    return predictions, predictions_with_weather

In [4]:
predictions, predictions_with_weather = forecast_next_24_hours_output_flow_rate(df, 2023, 8, 18, 13)

3673    36.07
Name: target_1, dtype: float64
Loading model xgb_1h.pkl to predict target_1
Loading model xgb_2h.pkl to predict target_2
Loading model xgb_3h.pkl to predict target_3
Loading model xgb_4h.pkl to predict target_4


ValueError: Feature shape mismatch, expected: 30, got 64